In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from logger import Logger, WanDBLogger

from dataset import TargetDataset, preprocess_data
from model import TransformerPredictor

In [3]:
# PARAMETERS
SIZE = 32
OBSERVATION_WINDOW = 5

data_runs = pd.read_csv('../data/dataset.csv')
data_runs = data_runs[~data_runs['action'].isna()]
data_scenarios = pd.read_csv('../data/scenarios.csv')
data = pd.merge(data_runs, data_scenarios[["layout", "scenario", "goals"]], on=["layout", "scenario"])
data = preprocess_data(data, OBSERVATION_WINDOW, SIZE)

In [4]:
nlayouts = len(data["layout"].unique())
train_data = data[data["layout"]<=nlayouts*0.8]
valid_data = data[data["layout"]>nlayouts*0.8]

# Datasets
train_dataset = TargetDataset(train_data, OBSERVATION_WINDOW)
valid_dataset = TargetDataset(valid_data, OBSERVATION_WINDOW)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [4]:
keys = ["epoch", "loss_train", "loss_valid"]
logger = WanDBLogger(keys)


# Dynamically determine input dimension
state_dim = 6
action_dim = len(data.iloc[0]['action_encoding'])
goals_dim = len(data.iloc[0]['goals_encoding'])
grid_dim = data["grid_encoding"][0].shape[0]
mlp_grid_dim = [grid_dim, 256, 64, 30]

goals_dim = len(data.iloc[0]['goals_encoding'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerPredictor(state_dim=state_dim, goals_dim=goals_dim, mlp_grid_dim=mlp_grid_dim, window=OBSERVATION_WINDOW,
                             num_layers=1, num_heads=4, output_dim=action_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training Loop
for epoch in range(60):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", leave=False)
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch["action"])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item(), avg_loss=total_loss / (progress_bar.n + 1))

    model.eval()
    val_loss = 0
    with torch.no_grad():
        progress_bar = tqdm(valid_dataloader, desc=f"Epoch {epoch+1}", leave=False)
        for batch in progress_bar:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch)
            loss = criterion(outputs, batch["action"]).item() 
            val_loss += loss 
            progress_bar.set_postfix(loss=val_loss, avg_loss=val_loss / (progress_bar.n + 1))
    
    epoch_log = {"epoch": epoch+1, "loss_train": total_loss/len(train_dataloader), "loss_valid": val_loss/len(valid_dataloader)}
    logger.log_epoch(epoch_log)


print("Training complete.")

2025/02/20 06:14:02 PM Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
2025/02/20 06:14:02 PM Starting new HTTPS connection (1): api.wandb.ai:443
2025/02/20 06:14:02 PM https://api.wandb.ai:443 "POST /graphql HTTP/11" 200 None
2025/02/20 06:14:03 PM https://api.wandb.ai:443 "POST /graphql HTTP/11" 200 None
wandb: Currently logged in as: cristian-rojascardenas (noteam2) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
2025/02/20 06:14:03 PM Popen(['git', 'cat-file', '--batch-check'], cwd=/home/crarojasca/Storage/Monash/DARPA/gr_pursuer, stdin=<valid stream>, shell=False, universal_newlines=False)


/home/crarojasca/Storage/anaconda3/envs/ansr/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/52 [00:00<?, ?it/s]

2025/02/20 06:14:09 PM epoch: 1.000000 | loss_train: 0.812363 | loss_valid: 0.771373 |


...

2025/02/20 06:19:19 PM epoch: 59.000000 | loss_train: 0.717786 | loss_valid: 0.760644 |


Epoch 60:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 60:   0%|          | 0/52 [00:00<?, ?it/s]

2025/02/20 06:19:24 PM epoch: 60.000000 | loss_train: 0.709318 | loss_valid: 0.768359 |


Training complete.


In [11]:
torch.save(model.state_dict(), f"models/{logger.run_name}.pth")